<a href="https://colab.research.google.com/github/pandian-ra/EVA4/blob/master/EVA4_Session_4_93-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size = 3) #26
        self.batch1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 16, kernel_size = 3) #24
        self.batch2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, kernel_size = 3) #22
        self.batch3 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2) #11
        self.conv4 = nn.Conv2d(32, 16, kernel_size = 1) #11
        self.batch4 = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(16, 16, kernel_size = 3) #9
        self.batch5 = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 16, kernel_size = 3) #7
        self.batch6 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2) #4
        self.conv7 = nn.Conv2d(16, 16, kernel_size = 1) #4
        self.batch7 = nn.BatchNorm2d(16)
        self.conv8 = nn.Conv2d(16, 32, kernel_size = 3) #2
        self.batch8 = nn.BatchNorm2d(32)
        # self.conv8 = nn.Conv2d(32, 32, kernel_size = 3) #3
        # self.conv9 = nn.Conv2d(32, 32, kernel_size = 3) #1
        self.conv10 = nn.Conv2d(32, 10, kernel_size = 1)
        # self.conv8 = nn.Conv2d(32, 32, 3) #0
        self.conv_drop1 = nn.Dropout2d(0.15)
        self.conv_drop2 = nn.Dropout2d(0.10)
        self.conv_drop3 = nn.Dropout2d(0.05)
        self.conv_drop4 = nn.Dropout2d(0.2)
        

    def forward(self, x):
        x = self.pool1(self.conv_drop3(F.relu(self.batch3(self.conv3(self.conv_drop2(F.relu(self.batch2(self.conv2(self.conv_drop3(F.relu(self.batch1(self.conv1(x)))))))))))))
        x = self.pool2(self.conv_drop3(F.relu(self.batch6(self.conv6(self.conv_drop2(F.relu(self.batch5(self.conv5(self.conv_drop3(F.relu(self.batch4(self.conv4(x)))))))))))))
        x = (F.relu(self.batch8(self.conv8(self.conv_drop3(F.relu(self.batch7(self.conv7(x))))))))

        # x = self.pool1(self.conv_drop3(F.relu(self.batch3(self.conv3(self.conv_drop2(F.relu(self.batch2(self.conv2(self.conv_drop3(F.relu(self.batch1(self.conv1(x)))))))))))))
        # x = self.pool2(self.conv_drop3(F.relu(self.batch6(self.conv6(self.conv_drop2(F.relu(self.batch5(self.conv5(self.conv_drop3(F.relu(self.batch4(self.conv4(x)))))))))))))
        # x = (F.relu(self.batch8(self.conv8(self.conv_drop3(F.relu(self.batch7(self.conv7(x))))))))
       

        # x = (self.conv9(x))
        # x = torch.flatten(x)
        x = (self.conv10(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
       BatchNorm2d-2           [-1, 32, 26, 26]              64
         Dropout2d-3           [-1, 32, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           4,624
       BatchNorm2d-5           [-1, 16, 24, 24]              32
         Dropout2d-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 32, 22, 22]           4,640
       BatchNorm2d-8           [-1, 32, 22, 22]              64
         Dropout2d-9           [-1, 32, 22, 22]               0
        MaxPool2d-10           [-1, 32, 11, 11]               0
           Conv2d-11           [-1, 16, 11, 11]             528
      BatchNorm2d-12           [-1, 16, 11, 11]              32
        Dropout2d-13           [-1, 16, 11, 11]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                    # transforms.ToPILImage(),
                    # transforms.RandomRotation(15),
                    transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                    transforms.ColorJitter(brightness=0.2, contrast=0.2),
                    
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0664, Accuracy: 9787/10000 (97.8700%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0497, Accuracy: 9847/10000 (98.4700%)



  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0298, Accuracy: 9903/10000 (99.0300%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0280, Accuracy: 9904/10000 (99.0400%)



loss=0.05304360389709473 batch_id=937: 100%|██████████| 938/938 [00:31<00:00, 29.81it/s]



Test set: Average loss: 0.0271, Accuracy: 9913/10000 (99.1300%)



In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.5)

for epoch in range(1, 5):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0242, Accuracy: 9915/10000 (99.1500%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0238, Accuracy: 9925/10000 (99.2500%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0244, Accuracy: 9918/10000 (99.1800%)



loss=0.14149799942970276 batch_id=937: 100%|██████████| 938/938 [00:31<00:00, 29.89it/s]



Test set: Average loss: 0.0240, Accuracy: 9923/10000 (99.2300%)



In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(1, 5):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0240, Accuracy: 9922/10000 (99.2200%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0229, Accuracy: 9926/10000 (99.2600%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0228, Accuracy: 9931/10000 (99.3100%)



loss=0.1568654477596283 batch_id=937: 100%|██████████| 938/938 [00:31<00:00, 30.02it/s]



Test set: Average loss: 0.0226, Accuracy: 9923/10000 (99.2300%)

